In [1]:
%load_ext autoreload
%autoreload 2

In [72]:
import random
from utils import *
from models import *
from attention import *

import torch
from torch.utils import data
from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
# Download the file
get_file()
path_to_file = 'spa-eng/spa.txt'

======== THE DATASET IS READY ========


In [41]:
# Try experimenting with the size of that dataset
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [42]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

24000 24000 6000 6000


In [43]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
2772 ----> salte
14 ----> de
13 ----> la
198 ----> cama
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
4 ----> i
4245 ----> sprang
74 ----> out
58 ----> of
218 ----> bed
3 ----> .
2 ----> <end>


In [44]:
input_tensor_train

array([[   1, 2772,   14, ...,    0,    0,    0],
       [   1,   63, 6470, ...,    0,    0,    0],
       [   1,    4,   90, ...,    0,    0,    0],
       ...,
       [   1,    4,   16, ...,    0,    0,    0],
       [   1,    6,   22, ...,    0,    0,    0],
       [   1,    9,  957, ...,    0,    0,    0]], dtype=int32)

In [45]:
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

In [46]:
# transform to torch tensor
tensor_x = torch.Tensor(input_tensor_train).long() 
tensor_y = torch.Tensor(target_tensor_train).long()
# create your datset
my_dataset = data.TensorDataset(tensor_x,tensor_y) 
# create your dataloader
my_dataloader = data.DataLoader(my_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        num_workers=4)   

In [47]:
my_dataset[20]

(tensor([   1,   19,  122,    7, 1582,    3,    2,    0,    0,    0,    0,    0,
            0,    0,    0,    0]),
 tensor([   1,   21,  139,    8, 1643,    3,    2,    0,    0,    0,    0]))

In [51]:
example_input_batch, example_target_batch = next(iter(my_dataloader))

In [52]:
example_input_batch.shape

torch.Size([64, 16])

In [55]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE, )

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) torch.Size([64, 16, 1024])
Encoder Hidden state shape: (batch size, units) torch.Size([1, 64, 1024])


In [60]:
example_input_batch.sum()

tensor(206749)

In [70]:
attention_layer = BahdanauAttention(10, 1024)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) torch.Size([64, 1024])
Attention weights shape: (batch_size, sequence_length, 1) torch.Size([64, 16, 1])


In [88]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, 1024)

sample_decoder_output, _, _ = decoder(torch.randint(1, 20, (BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) torch.Size([64, 4935])
